# Aplicação do YOLO Tradicional

Este notebook aplica o modelo YOLO tradicional (pré-treinado na COCO) às nossas imagens de teste e compara seu desempenho com os modelos YOLO customizados treinados anteriormente.

## 1. Configuração do Ambiente

Primeiro, vamos importar as bibliotecas necessárias e configurar o ambiente.

In [ ]:
# Verificar se o ambiente já foi configurado
import os
import sys

# Se o ambiente ainda não foi configurado, execute o setup_env.sh
if not os.path.exists('../yolov5'):
    print("Configurando o ambiente com setup_env.sh...")
    !chmod +x ../setup_env.sh
    !../setup_env.sh
else:
    print("Ambiente já configurado.")

# Importar bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torch
import yaml
import time
from pathlib import Path
from tqdm.notebook import tqdm

# Adicionar o diretório YOLOv5 ao path
yolov5_path = os.path.abspath('../yolov5')
if yolov5_path not in sys.path:
    sys.path.insert(0, yolov5_path)
    print(f"Adicionado {yolov5_path} ao início do sys.path")

# Adicionar os subdiretórios do YOLOv5 ao path
sys.path.append(os.path.join(yolov5_path, 'models'))
sys.path.append(os.path.join(yolov5_path, 'utils'))

## 2. Carregamento dos Modelos

Vamos carregar o modelo YOLO tradicional pré-treinado e os modelos YOLO customizados para comparação.

In [ ]:
# Importar funções necessárias do YOLOv5
from models.experimental import attempt_load
from utils.general import non_max_suppression
from utils.plots import Annotator, colors

# Carregar o modelo YOLO tradicional pré-treinado
model_traditional = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
print("Modelo YOLO tradicional carregado.")

# Caminhos para os modelos customizados
model_custom_path = '../models/yolo_custom_30epochs/weights/best.pt'

# Verificar se o modelo customizado existe
if os.path.exists(model_custom_path):
    # Carregar o modelo customizado
    model_custom = attempt_load(model_custom_path)
    model_custom.eval()
    print("Modelo YOLO customizado carregado.")
    
    # Carregar o arquivo data.yaml para obter as categorias customizadas
    if os.path.exists('../data/data.yaml'):
        with open('../data/data.yaml', 'r') as f:
            data_yaml = yaml.safe_load(f)
        custom_categories = data_yaml['names']
        print(f"Categorias customizadas: {custom_categories}")
    else:
        print("Arquivo data.yaml não encontrado. Usando categorias padrão.")
        custom_categories = ['apple', 'banana']
else:
    print(f"Modelo customizado não encontrado: {model_custom_path}")
    model_custom = None
    custom_categories = ['apple', 'banana']

# Obter as categorias do COCO (usadas pelo YOLO tradicional)
coco_categories = model_traditional.names
print(f"\nCategorias COCO (primeiras 10): {list(coco_categories.values())[:10]}...")

## 3. Mapeamento de Categorias

Vamos mapear nossas categorias customizadas para as categorias correspondentes no COCO, se possível.

In [ ]:
# Função para encontrar a categoria COCO mais próxima
def find_closest_coco_category(custom_category, coco_categories):
    """Encontra a categoria COCO mais próxima da categoria customizada."""
    # Implementação simples: procura por correspondência exata ou parcial
    custom_category = custom_category.lower()
    
    # Tentar correspondência exata
    for idx, coco_cat in coco_categories.items():
        if custom_category == coco_cat.lower():
            return idx, coco_cat
    
    # Tentar correspondência parcial
    for idx, coco_cat in coco_categories.items():
        if custom_category in coco_cat.lower() or coco_cat.lower() in custom_category:
            return idx, coco_cat
    
    # Se não encontrar, retornar None
    return None, None

# Mapear categorias customizadas para categorias COCO
category_mapping = {}
for i, custom_cat in enumerate(custom_categories):
    coco_idx, coco_cat = find_closest_coco_category(custom_cat, coco_categories)
    category_mapping[i] = (coco_idx, coco_cat)
    print(f"Categoria customizada '{custom_cat}' mapeada para categoria COCO: {coco_cat} (ID: {coco_idx})")

# Se não encontrou correspondência, permitir que o usuário defina manualmente
for i, (coco_idx, coco_cat) in category_mapping.items():
    if coco_idx is None:
        print(f"\nNão foi possível mapear automaticamente a categoria '{custom_categories[i]}'")
        print("Por favor, selecione uma categoria COCO correspondente:")
        
        # Mostrar algumas categorias COCO comuns
        common_categories = [0, 1, 2, 3, 5, 7, 9, 15, 16, 17, 18, 19, 20, 39, 41, 60, 62, 63, 67, 73, 76]
        for idx in common_categories:
            print(f"  {idx}: {coco_categories[idx]}")
        
        # Permitir entrada manual (em um ambiente real)
        # coco_idx = int(input("Digite o ID da categoria COCO: "))
        # category_mapping[i] = (coco_idx, coco_categories[coco_idx])
        
        # Para este notebook, vamos definir um valor padrão
        if custom_categories[i].lower() == 'apple':
            coco_idx = 47  # apple
        elif custom_categories[i].lower() == 'banana':
            coco_idx = 52  # banana
        else:
            coco_idx = 0   # pessoa (padrão)
            
        category_mapping[i] = (coco_idx, coco_categories[coco_idx])
        print(f"Categoria '{custom_categories[i]}' mapeada para '{coco_categories[coco_idx]}' (ID: {coco_idx})")

## 4. Funções para Detecção e Visualização

Vamos definir funções para detectar objetos e visualizar os resultados.

In [ ]:
# Função para detectar objetos com o modelo YOLO tradicional
def detect_traditional(model, image_path, conf_threshold=0.25):
    """Detecta objetos em uma imagem usando o modelo YOLO tradicional."""
    # Carregar a imagem
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Medir o tempo de inferência
    start_time = time.time()
    
    # Fazer a predição
    results = model(img)
    
    # Calcular o tempo de inferência
    inference_time = time.time() - start_time
    
    # Filtrar por confiança
    results.xyxy[0] = results.xyxy[0][results.xyxy[0][:, 4] >= conf_threshold]
    
    return img, results, inference_time

# Função para detectar objetos com o modelo YOLO customizado
def detect_custom(model, image_path, conf_threshold=0.25):
    """Detecta objetos em uma imagem usando o modelo YOLO customizado."""
    # Carregar a imagem
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Redimensionar a imagem para 640x640 (tamanho esperado pelo modelo)
    img_resized = cv2.resize(img, (640, 640))
    
    # Preparar a imagem para o modelo
    img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float().div(255.0).unsqueeze(0)
    
    # Medir o tempo de inferência
    start_time = time.time()
    
    # Fazer a predição
    with torch.no_grad():
        pred = model(img_tensor)[0]
        pred = non_max_suppression(pred, conf_threshold)
    
    # Calcular o tempo de inferência
    inference_time = time.time() - start_time
    
    # Processar as predições
    results = []
    for det in pred:
        if len(det):
            # Ajustar as coordenadas para a imagem original
            h, w = img.shape[:2]
            scale_x, scale_y = w / 640, h / 640
            
            for *xyxy, conf, cls in det:
                # Converter as coordenadas para a escala da imagem original
                x1, y1, x2, y2 = xyxy
                x1 = int(x1.item() * scale_x)
                y1 = int(y1.item() * scale_y)
                x2 = int(x2.item() * scale_x)
                y2 = int(y2.item() * scale_y)
                
                results.append(([x1, y1, x2, y2], conf.item(), int(cls.item())))
    
    return img, results, inference_time

# Função para visualizar as detecções do modelo tradicional
def visualize_traditional(img, results):
    """Visualiza as detecções do modelo YOLO tradicional."""
    # Criar uma cópia da imagem
    annotated_img = img.copy()
    
    # Desenhar as detecções
    for *xyxy, conf, cls_id in results.xyxy[0]:
        # Converter para inteiros
        x1, y1, x2, y2 = map(int, xyxy)
        cls_id = int(cls_id.item())
        conf = conf.item()
        
        # Obter a categoria
        category = results.names[cls_id]
        
        # Desenhar o retângulo
        color = colors(cls_id, True)
        cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color, 2)
        
        # Adicionar o rótulo
        label = f"{category} {conf:.2f}"
        t_size = cv2.getTextSize(label, 0, fontScale=0.5, thickness=1)[0]
        cv2.rectangle(annotated_img, (x1, y1 - t_size[1] - 3), (x1 + t_size[0], y1), color, -1)
        cv2.putText(annotated_img, label, (x1, y1 - 2), 0, 0.5, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
    
    return annotated_img

# Função para visualizar as detecções do modelo customizado
def visualize_custom(img, results, class_names):
    """Visualiza as detecções do modelo YOLO customizado."""
    # Criar uma cópia da imagem
    annotated_img = img.copy()
    
    # Desenhar as detecções
    for xyxy, conf, cls in results:
        x1, y1, x2, y2 = xyxy
        label = f"{class_names[cls]} {conf:.2f}"
        
        # Desenhar o retângulo
        color = colors(cls, True)
        cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color, 2)
        
        # Adicionar o rótulo
        t_size = cv2.getTextSize(label, 0, fontScale=0.5, thickness=1)[0]
        cv2.rectangle(annotated_img, (x1, y1 - t_size[1] - 3), (x1 + t_size[0], y1), color, -1)
        cv2.putText(annotated_img, label, (x1, y1 - 2), 0, 0.5, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
    
    return annotated_img

## 5. Aplicação dos Modelos nas Imagens de Teste

Vamos aplicar os modelos YOLO tradicional e customizado nas imagens de teste e comparar os resultados.

In [ ]:
# Obter imagens de teste
test_img_dir = '../dataset/test/images'
test_img_files = [os.path.join(test_img_dir, f) for f in os.listdir(test_img_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.avif'))]

# Verificar se há imagens de teste
if not test_img_files:
    print(f"Nenhuma imagem de teste encontrada em {test_img_dir}")
else:
    print(f"Encontradas {len(test_img_files)} imagens de teste.")
    
    # Selecionar algumas imagens para visualização
    import random
    random.seed(42)  # Para reprodutibilidade
    sample_imgs = random.sample(test_img_files, min(4, len(test_img_files)))
    
    # Listas para armazenar tempos de inferência
    inference_times_traditional = []
    inference_times_custom = []
    
    # Aplicar os modelos nas imagens selecionadas
    plt.figure(figsize=(15, 12))
    
    for i, img_path in enumerate(sample_imgs):
        try:
            print(f"Processando imagem: {img_path}")
            
            # Modelo YOLO tradicional
            img_trad, results_trad, time_trad = detect_traditional(model_traditional, img_path)
            annotated_img_trad = visualize_traditional(img_trad, results_trad)
            inference_times_traditional.append(time_trad)
            
            # Modelo YOLO customizado (se disponível)
            if model_custom is not None:
                img_custom, results_custom, time_custom = detect_custom(model_custom, img_path)
                annotated_img_custom = visualize_custom(img_custom, results_custom, custom_categories)
                inference_times_custom.append(time_custom)
            else:
                # Se o modelo customizado não estiver disponível, mostrar a imagem original
                annotated_img_custom = img_trad.copy()
                time_custom = 0
            
            # Mostrar as imagens lado a lado
            plt.subplot(len(sample_imgs), 2, i*2+1)
            plt.imshow(annotated_img_trad)
            plt.title(f"YOLO Tradicional - {os.path.basename(img_path)} - {time_trad:.3f}s")
            plt.axis('off')
            
            plt.subplot(len(sample_imgs), 2, i*2+2)
            plt.imshow(annotated_img_custom)
            plt.title(f"YOLO Customizado - {os.path.basename(img_path)} - {time_custom:.3f}s")
            plt.axis('off')
        except Exception as e:
            print(f"Erro ao processar a imagem {img_path}: {e}")
            import traceback
            traceback.print_exc()
    
    plt.tight_layout()
    plt.show()
    
    # Calcular tempos médios de inferência
    if inference_times_traditional:
        avg_time_trad = np.mean(inference_times_traditional)
        print(f"Tempo médio de inferência (YOLO Tradicional): {avg_time_trad:.4f} segundos")
    
    if inference_times_custom:
        avg_time_custom = np.mean(inference_times_custom)
        print(f"Tempo médio de inferência (YOLO Customizado): {avg_time_custom:.4f} segundos")
        print(f"Diferença: {(avg_time_custom - avg_time_trad) / avg_time_trad * 100:.2f}%")

## 6. Avaliação Quantitativa

Vamos avaliar quantitativamente o desempenho dos modelos YOLO tradicional e customizado.

In [ ]:
# Função para avaliar o modelo YOLO tradicional no conjunto de teste
def evaluate_traditional_model():
    """Avalia o modelo YOLO tradicional no conjunto de teste."""
    print("Avaliando o modelo YOLO tradicional...")
    
    # Filtrar apenas as classes que nos interessam (mapeadas das nossas categorias customizadas)
    classes = [coco_idx for coco_idx, _ in category_mapping.values() if coco_idx is not None]
    
    # Verificar se temos classes válidas
    if not classes:
        print("Nenhuma classe COCO válida encontrada para avaliação.")
        return
    
    # Executar a validação sem a opção --classes que está causando erro
    print(f"Avaliando classes COCO: {classes}")
    !cd ../yolov5 && python val.py --weights yolov5s.pt --data coco.yaml --img 640 --batch 16 --task test

# Função para avaliar o modelo YOLO customizado no conjunto de teste
def evaluate_custom_model(model_path):
    """Avalia o modelo YOLO customizado no conjunto de teste."""
    print(f"Avaliando o modelo YOLO customizado: {model_path}")
    
    # Verificar se o arquivo de pesos existe
    if not os.path.exists(model_path):
        print(f"❌ Arquivo de pesos não encontrado: {model_path}")
        # Tentar encontrar o arquivo em um caminho absoluto
        abs_path = os.path.abspath(model_path)
        print(f"Tentando caminho absoluto: {abs_path}")
        if os.path.exists(abs_path):
            model_path = abs_path
            print(f"✅ Arquivo de pesos encontrado: {model_path}")
        else:
            print(f"❌ Arquivo de pesos não encontrado mesmo com caminho absoluto.")
            return
    
    # Verificar se o arquivo data.yaml existe
    data_yaml_path = '../data/data.yaml'
    if not os.path.exists(data_yaml_path):
        print(f"❌ Arquivo data.yaml não encontrado: {data_yaml_path}")
        return
    
    # Executar a validação
    !cd ../yolov5 && python val.py --weights {model_path} --data ../data/data.yaml --img 640 --batch 16 --task test

# Avaliar os modelos
if os.path.exists('../dataset/test/labels'):
    # Verificar se há arquivos de anotação no conjunto de teste
    label_files = [f for f in os.listdir('../dataset/test/labels') if f.endswith('.txt')]
    if label_files:
        print(f"Encontrados {len(label_files)} arquivos de anotação no conjunto de teste.")
        
        # Avaliar o modelo YOLO customizado (se disponível)
        if model_custom is not None:
            # Usar o caminho absoluto para o modelo customizado
            model_custom_abs_path = os.path.abspath(model_custom_path)
            print(f"Caminho absoluto para o modelo customizado: {model_custom_abs_path}")
            evaluate_custom_model(model_custom_abs_path)
        
        # Avaliar o modelo YOLO tradicional
        # Nota: Esta avaliação pode não ser precisa, pois o modelo tradicional foi treinado em COCO
        # e nossas anotações são específicas para nossas categorias customizadas
        print("\nNota: A avaliação do modelo YOLO tradicional pode não ser precisa devido às diferenças nas categorias.")
        evaluate_traditional_model()
    else:
        print("Nenhum arquivo de anotação encontrado no conjunto de teste.")
else:
    print("Diretório de anotações de teste não encontrado.")

## 7. Análise Comparativa

Vamos analisar e comparar os resultados dos modelos YOLO tradicional e customizado.

### Comparação de Desempenho

Com base nos resultados obtidos, podemos comparar o desempenho do YOLO tradicional e do YOLO customizado:

1. **Precisão na Detecção**:
   - O YOLO tradicional foi treinado em 80 categorias do COCO, enquanto nosso modelo customizado foi treinado especificamente em nossas categorias de interesse.
   - O modelo customizado tende a ser mais preciso para nossas categorias específicas, enquanto o modelo tradicional pode detectar uma variedade maior de objetos, mas com menor precisão para nossas categorias de interesse.

2. **Tempo de Inferência**:
   - Ambos os modelos usam a mesma arquitetura (YOLOv5s), então o tempo de inferência deve ser similar.
   - Pequenas diferenças podem ocorrer devido ao número de classes (80 no tradicional vs. 2 no customizado).

3. **Facilidade de Uso**:
   - O YOLO tradicional é mais fácil de usar, pois não requer treinamento específico.
   - O YOLO customizado requer coleta de dados, anotação e treinamento, mas oferece melhor desempenho para casos específicos.

4. **Adaptabilidade**:
   - O YOLO tradicional é limitado às 80 categorias do COCO.
   - O YOLO customizado pode ser adaptado para detectar qualquer objeto, desde que haja dados de treinamento suficientes.

### Conclusões

1. **Quando usar o YOLO tradicional**:
   - Quando as categorias de interesse estão bem representadas no COCO.
   - Quando não há tempo ou recursos para coletar dados e treinar um modelo customizado.
   - Quando é necessário detectar uma variedade de objetos diferentes.

2. **Quando usar o YOLO customizado**:
   - Quando as categorias de interesse são específicas e não estão bem representadas no COCO.
   - Quando é necessário alta precisão para categorias específicas.
   - Quando há dados de treinamento suficientes disponíveis.

No próximo notebook, vamos treinar uma CNN do zero para classificação e comparar seu desempenho com as abordagens YOLO.